In [49]:
import os;
import numpy as np;
from tensorflow.keras.preprocessing.text import Tokenizer;
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"#选择哪一块gpu,-1代表cpu

In [50]:
# 载入文档
def load_documents(path):
    files =[];
    for filename in os.listdir(path):
        path_of_file = path + '/' + filename;
        if os.path.getsize(path_of_file) > 0:
            if len(filename) > 3 and filename[-3:] == 'txt':
                f = open(path_of_file,'r');
                content = f.read();
                files.append(content);
                f.close();
            else:
                print('file <\'' + filename + '\'> was ignored for <txt> file only');
        else:
            print('file <\'' + filename + '\'> was ignored for its zero size');
    print('done, while Getting {} documents loaded'.format(len(files)))
    return files;
# 文档目录，注意修改。
path = '/home/pp21/data/document';
document = load_documents(path);
# '''2个名为 <No*> 的文件为验证上述函数泛化能力的空文件'''

file <'NO'> was ignored for <txt> file only
file <'Nothing.txt'> was ignored for its zero size
done, while Getting 8 documents loaded


In [51]:
# 载入停顿词
path = '/home/pp21/data/stop_words/stop words.txt';
f = open(path,'r',);
stop_words = f.read();
f.close();
stop_words = stop_words.split('\n');

In [52]:
# 载入实体
stop_words += ['sam','michelle','lele'];

In [53]:
num_words = 100;
num_documents = len(document);
num_topics = 5;
tokenizer = Tokenizer(num_words=num_words);# 内容量为100的词典
tokenizer.fit_on_texts(document);# 以频率次数排列词典
word_index = tokenizer.word_index;
words = list(word_index.keys());

In [54]:
def remove_stopwords(words,num_words,stop_words):
    len_words = len(words);
    t = 0;
    n = 0;
    selected_words = [];
    while t < len_words and n < num_words:
        if not words[t] in stop_words:
            selected_words.append(words[t]);
            n += 1;
        t += 1;
    if len(selected_words) < num_words:
        print('quantities of selected words were not Enough; Get/Require: {}/{}'.format(len(selected_words),num_words));
    return selected_words;

In [55]:
# 去掉停用词
def remove_stopwords(words,num_words,stop_words):
    len_words = len(words);
    t = 0;
    n = 0;
    selected_words = [];
    while t < len_words and (n < num_words or num_words < 0):
        if not words[t] in stop_words:
            selected_words.append(words[t]);
            n += 1;
        t += 1;
    if len(selected_words) < num_words:
        print('quantities of selected words were not Enough; Get/Require: {}/{}'.format(len(selected_words),num_words));
    return selected_words;
words = remove_stopwords(words,-1,stop_words);

In [56]:
# 获取TF IDF TFIDF 词频 单词
def get_TFIDF(texts,words,num_words=100,more_info=False,stop_words=None):
    tokenizer = Tokenizer();# 内容量为100的词典
    tokenizer.fit_on_texts(texts);# 以频率次数排列词典
    word_index = tokenizer.word_index;
    word_counts = tokenizer.word_counts;
    TF = [];
    F = [];
    for text in texts:
        this_tokenizer = Tokenizer();
        this_tokenizer.fit_on_texts([text]);
        this_word_counts = this_tokenizer.word_counts;
        this_words = list(this_tokenizer.word_index.keys());
        f = [];
        t_frequence = [];
        for word in words:
            if word in this_words:
                fre = this_word_counts[word];
                t_fre = fre/len(this_words);
                f.append(fre);
                t_frequence.append(t_fre);
            else:
                f.append(0);
                t_frequence.append(0);
        TF.append(t_frequence);
        F.append(f);
    F = np.array(F);
    TF = np.array(TF);
    DF = [];
    for word in words:
        df = word_counts[word];
        DF.append(df);
    DF = np.array(DF).reshape(-1,1);
    IDF = np.log10(len(texts)/(1+DF));
    TFIDF = TF.T*IDF;
    if more_info == True:
        return F.T,TF.T,DF,TFIDF;
    else:
        return TFIDF;
# 只需要TFIDF则考虑 more_info = False（默认）
F,TF,DF,TFIDF = get_TFIDF(texts=document,words=words,more_info=True,stop_words=stop_words);

In [57]:
num_words = len(words)
# 初始化概率矩阵（概率分布）
n_wd_matrix = F;
p_zd_matrix = np.abs(np.random.randn(num_topics,num_documents));
p_wz_matrix = np.abs(np.random.randn(num_words,num_topics));
p_zwd_matrix = np.abs(np.random.randn(num_topics,num_words,num_documents));
p_zd_matrix = p_zd_matrix/np.max(p_zd_matrix);
p_wz_matrix = p_wz_matrix/np.max(p_wz_matrix);
p_zwd_matrix = p_zwd_matrix/np.max(p_zwd_matrix);

In [58]:
# 矩阵运算
import copy;
epochs = 20;
for e in range(epochs):
    last_zd_m = copy.deepcopy(p_zd_matrix);
    '''那个sb玩意儿搞得浅copy？？？？？我特么缺你这点内存吗？？？？'''
    # E-step
    for k in range(num_topics):
        p_zwd_matrix[k,:,:] = np.dot(p_wz_matrix[:,k:k+1],p_zd_matrix[k:k+1,:])/np.dot(p_wz_matrix,p_zd_matrix);
    L = np.sum(n_wd_matrix*np.sum(p_zwd_matrix*np.log(np.dot(p_wz_matrix,p_zd_matrix)),axis=0));
    # M-step
    for i in range(num_documents):
        p_zk = np.dot(p_zwd_matrix[:,:,i],n_wd_matrix[:,i]);
        p_zd_matrix[:,i] = p_zk/np.sum(p_zk);
    p_wj = [];
    for j in range(num_words):
        res = np.dot(p_zwd_matrix[:,j,:],n_wd_matrix[j,:])
        p_wj.append(res);
    p_wj = np.array(p_wj);
    p_wz_matrix = p_wj/np.sum(p_wj,axis=0);
    delta = np.max(np.abs(last_zd_m-p_zd_matrix));# 迭代中绝对值最大误差（无穷范数）
    print('{:d}\tdelta*100: {:.8f}\tL: {:.8f}'.format(e+1,delta*100,L))

1	delta*100: 49.48880313	L: -613.38590764
2	delta*100: 23.97570140	L: -2757.22216227
3	delta*100: 24.49180222	L: -2631.97410037
4	delta*100: 20.87397432	L: -2483.53678879
5	delta*100: 13.55119721	L: -2367.61202140
6	delta*100: 10.26496096	L: -2307.51018582
7	delta*100: 11.58536242	L: -2270.82544486
8	delta*100: 12.71257686	L: -2236.38812272
9	delta*100: 10.63705744	L: -2204.45944596
10	delta*100: 5.25763524	L: -2180.31968912
11	delta*100: 5.51413136	L: -2167.55082931
12	delta*100: 5.88348786	L: -2159.93531858
13	delta*100: 5.79929027	L: -2150.87976672
14	delta*100: 4.89542754	L: -2141.09423088
15	delta*100: 2.98813064	L: -2131.56563480
16	delta*100: 1.11915369	L: -2124.97592197
17	delta*100: 0.63762437	L: -2122.07276862
18	delta*100: 0.46808474	L: -2119.87498884
19	delta*100: 0.07563673	L: -2118.81188713
20	delta*100: 0.00488951	L: -2118.70092698


In [59]:
# 打印文档/主题概率分布
def print_topic(m):
    print('topic of documents:');
    print('document\\topic\t',end='')
    for i in range(m.shape[0]):
        print(' {} \t'.format(i+1),end='');
        pass;
    print('',end='\n');
    print(' '*8*2,end='');
    for i in range(m.shape[0]):
        print('-'*8,end='');
        pass;
    print('',end='\n');
    for i in range(m.shape[1]):
        print('\t{}    |\t'.format(i+1),end='')
        for j in range(m.shape[0]):
            if m[j,i] < 1e-4:
                print(' 0\t',end='')
            else:
                print(' {:.4f}\t'.format(m[j,i]),end='');
        print('',end='\n');
print_topic(p_zd_matrix);

topic of documents:
document\topic	 1 	 2 	 3 	 4 	 5 	
                ----------------------------------------
	1    |	 0	 1.0000	 0	 0	 0	
	2    |	 0	 0	 0	 0	 1.0000	
	3    |	 0	 0	 0	 1.0000	 0	
	4    |	 0	 1.0000	 0	 0	 0	
	5    |	 0	 0	 1.0000	 0	 0	
	6    |	 0	 0	 0	 0	 1.0000	
	7    |	 1.0000	 0	 0	 0	 0	
	8    |	 0	 0	 1.0000	 0	 0	


In [60]:
# 打印单词/主题概率分布
def print_words_of_topic(m,n_words,words):
    print('words of topic:');
    print('topic\\words\t',end='')
    for i in range(min(m.shape[0],n_words)):
        print(' {} \t\t'.format(i+1),end='');
        pass;
    print('',end='\n');
    for i in range(m.shape[1]):
        m_i = list(m[:,i]);
        sorted_nums = sorted(enumerate(m_i), key=lambda x: x[1])
        idx = [j[0] for j in sorted_nums]
        nums = [j[1] for j in sorted_nums]
        print('\t{}:\t'.format(i+1),end='')
        for j in range(min(m.shape[0],n_words)):
            word = words[idx[j]];
            print(word,end='');
            if len(word)<8:
                print('\t\t',end='');
            else:
                print('\t',end='');
        print('',end='\n');
print_words_of_topic(m=p_wz_matrix,n_words=6,words=words);
# n_words: 最大单词显示数量
# 该函数依概率打印前 n_words 个单词

words of topic:
topic\words	 1 		 2 		 3 		 4 		 5 		 6 		
	1:	climb		mountain	energy		meant		faith		radio		
	2:	earthquake	tv		house		lights		scared		damage		
	3:	falls		fire		field		fragrance	stays		carries		
	4:	climb		mountain	energy		meant		faith		mountains	
	5:	fish		bacteria	red		day		snack		hole		
